In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/MyDrive/Stage PFE - VERMEG/Data/cleaned_data.csv'
# file_path = 'cleaned_data.csv'
df = pd.read_csv(file_path)
df.head()

,Comment,Comment.1,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6,Comment.7,Comment.8,Comment.9,...,Comment.52,Comment.53,Comment.54,Comment.55,Comment.56,Comment.57,Comment.58,Environment,Summary,Description
0,I am running the command provided by the docum...,Hi [~accountid:6213cf7cc345490071968be0] The ...,"Ok, thanks for your reply. \n\nI had seen this...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,version we are running is: Artifactory Online ...,plugin config-import not found for jfrog Artif...,I am following the steps from [https://docs.jf...
1,[https://jfrog.com/help/r/jfrog-release-inform...,*According to RTDEV-5069:*\n\nThe feature is s...,[~accountid:5f79a75eac3a2d006ffcc8b7] : \n\n* ...,[~accountid:5f79a75eac3a2d006ffcc8b7] We have ...,Since Composer *Virtual* Repository supports o...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RTDEV-5069 / mirroring https://asset-packagist...,We are running Artifactory 7.63.14 and have pr...
2,This is the current value I would like to chan...,[https://jfrog.atlassian.net/browse/RTFACT-154...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Developement and Production,Docker - Repository Default URL,Similar to Issue: [https://jfrog.atlassian.net...
3,Ik can get the manifest from the remote repo w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Artifactory-pro 7.63.14 within a docker conta...,getting manifest unknown error pulling image,I am getting the following error when pulling ...
4,Corresponding Log record from nginx reverse pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Linux centos,Caught exception while saving incoming stream ...,We have recently upgrade Artifactory to 7.63.1...


In [6]:
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [9]:
df['Summary'] = df['Summary'].astype(str)

In [11]:
embeddings = []

# Iterate through each row of the column
for i in df['Summary']:
    # Encode the sentence using the USE model
    embedding = use_model([i])
    # Append the embedding to the list
    embeddings.append(embedding)

df['embeddingsSummary'] = embeddings

In [14]:
df['embeddingsSummary'][1].shape

TensorShape([1, 512])

In [12]:
from sklearn.metrics.pairwise import cosine_similarity


In [15]:
df['Summary'][700]

'Authentication  problem'

In [16]:
df['Description'][700]

'Hi,\nwe had some problems with our artifactory when the user access a virtual repo which has an local repo who you need an authenticated user.\n\n\n* virtual repo\n**  local repo 1\n**  local repo 2\n**  external repo 1\n** external repo 2\n\t\n\nPermissions Management:\n - Any Remote (on any remote repo): anonymous -> deploy\n - local-repo1-permission (local repo 1): reader-user -> read, deployer-user -> delete, deploy, annotate, read\n - local-repo2-permission (local repo 2): reader-user -> read, deployer-user -> read\n\nThe problem occur when the needed jar is in "local repo 1" and your credential are from one of the both users'

In [17]:
def clean_comment(comment):
  try:
    start_index = comment.index('[') + 1
    end_index = comment.rfind(']') + 1  # Use rfind for last closing bracket
    # return comment[start_index:end_index].strip()  # Remove leading/trailing whitespace
    return comment[end_index:].strip()  # Remove leading/trailing whitespace

  except ValueError:  # Handle cases where there's no list
    return comment


In [18]:
# Iterate through comment columns (replace 'Comment_1', 'Comment_2', etc. with your actual column names)
for col in df.filter(like='Comment'):
  df[col] = df[col].astype(str)
  df[col] = df[col].apply(clean_comment)

In [19]:
df.head()

,Comment,Comment.1,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6,Comment.7,Comment.8,Comment.9,...,Comment.53,Comment.54,Comment.55,Comment.56,Comment.57,Comment.58,Environment,Summary,Description,embeddingsSummary
0,I am running the command provided by the docum...,,or the command “jf plugin install config-impor...,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,version we are running is: Artifactory Online ...,plugin config-import not found for jfrog Artif...,I am following the steps from [https://docs.jf...,"((tf.Tensor(0.03961963, shape=(), dtype=float3..."
1,for Composer Remote repositories.{quote}\n\nRT...,it that relevant to the situation you described?,"does not support v2, only v1. We will check…",only supports v1. Can you explain the comment ...,Since Composer *Virtual* Repository supports o...,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,NaN,RTDEV-5069 / mirroring https://asset-packagist...,We are running Artifactory 7.63.14 and have pr...,"((tf.Tensor(0.008936056, shape=(), dtype=float..."
2,This is the current value I would like to chan...,,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,Developement and Production,Docker - Repository Default URL,Similar to Issue: [https://jfrog.atlassian.net...,"((tf.Tensor(0.020218397, shape=(), dtype=float..."
3,},nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,Artifactory-pro 7.63.14 within a docker conta...,getting manifest unknown error pulling image,I am getting the following error when pulling ...,"((tf.Tensor(0.004147171, shape=(), dtype=float..."
4,"""PUT ‘REPO_PATH/s.zip’ HTTP/1.1"" 400 0 ""-"" ""py...",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,Linux centos,Caught exception while saving incoming stream ...,We have recently upgrade Artifactory to 7.63.1...,"((tf.Tensor(-0.024265409, shape=(), dtype=floa..."


In [20]:
sum_input_sen = df['Summary'][700]
des_input_sen = df['Description'][700]

In [21]:
df["embeddingsSummary"] = df["embeddingsSummary"].tolist()

In [24]:
# Function to calculate cosine similarity between sentences
def calculate_similarity(input_sentence, embeddings):
  input_embedding = model([input_sentence])

  similarities = []
  for embedding in embeddings:
    similarity = cosine_similarity(input_embedding, embedding)[0][0]
    similarities.append(similarity)

  return similarities

In [25]:
# Calculate similarity scores
sum_similarities = calculate_similarity(sum_input_sen, df["embeddingsSummary"])

# Display results
for i, row in df.iterrows():
  Summary = row["Summary"]
  similarity = sum_similarities[i]
  print(f"Summary: {Summary}, Similarity: {similarity:.4f}")

Streaming output truncated to the last 5000 lines.
Summary: HV-1755 introduces NPE in org.hibernate.validator.internal.engine.ValidatorFactoryImpl constructor, Similarity: 0.2024
Summary: Hibernate Validation fails with JDK15, Similarity: 0.2709
Summary: Release 6.1.6, Similarity: 0.2055
Summary: Hibernate-validator contains high severity vulnerability, Similarity: 0.1221
Summary: @Min and @DecimalMin fail on BigDecimal.valueOf(Double.MIN_VALUE), Similarity: 0.1989
Summary: Validation on classes with a bidirectional relationship cause stack overflow on 6.1.x, Similarity: 0.1602
Summary: Optional containing a list of items is not being validated, Similarity: 0.1888
Summary: Missing char sequence implementation, Similarity: 0.1483
Summary: hibernate-validator unnecessarily calls the thread-unsafe CachingJPATraversableResolverForSingleValidation in hibernate-core when validating, Similarity: 0.1576
Summary: Invalid parsing of EL expression can lead to invalid EL expressions considered val

In [26]:
len(sum_similarities)

9419

In [27]:
# Store summaries with similarity > 0.7 and their scores in a list
similar_summaries_and_scores = []
for summary, similarity in zip(df["Summary"], sum_similarities):
  if similarity > 0.5:
    similar_summaries_and_scores.append((summary, similarity))

# Sort similar summaries by similarity score (highest first)
sorted_similar_summaries_and_scores = sorted(similar_summaries_and_scores, key=lambda x: x[1], reverse=True)

# Print top 5 similar sentences and their scores
for i, (summary, similarity) in enumerate(sorted_similar_summaries_and_scores):
  if len(sum_similarities) == 0:
    print('No similar tickets found.')
  if i == 5:
    break  # Print only the top 5
  print(f"Similar summary: {summary} \n , Similarity: {similarity:.4f} \n")

Similar summary: Authentication  problem 
 , Similarity: 1.0000 

Similar summary: Authentication 
 , Similarity: 0.7818 

Similar summary: Random authentification Issues 
 , Similarity: 0.6995 

Similar summary: Cannot log-in 
 , Similarity: 0.6109 

Similar summary: Invalid authentication removes groups from Crowd-users 
 , Similarity: 0.5257 



In [28]:
len(similar_summaries_and_scores)

7

In [29]:
def merge_comments(df):

  comment_cols = ['Comment', 'Comment.1', 'Comment.2', 'Comment.3', 'Comment.4', 'Comment.5', 'Comment.6', 'Comment.7', 'Comment.8', 'Comment.9', 'Comment.10', 'Comment.11', 'Comment.12', 'Comment.13', 'Comment.14', 'Comment.15', 'Comment.16', 'Comment.17', 'Comment.18', 'Comment.19', 'Comment.20', 'Comment.21', 'Comment.22', 'Comment.23', 'Comment.24', 'Comment.25', 'Comment.26', 'Comment.27', 'Comment.28', 'Comment.29', 'Comment.30', 'Comment.31', 'Comment.32', 'Comment.33', 'Comment.34', 'Comment.35', 'Comment.36', 'Comment.37', 'Comment.38', 'Comment.39', 'Comment.40', 'Comment.41', 'Comment.42', 'Comment.43', 'Comment.44', 'Comment.45', 'Comment.46', 'Comment.47', 'Comment.48', 'Comment.49', 'Comment.50', 'Comment.51', 'Comment.52', 'Comment.53', 'Comment.54', 'Comment.55', 'Comment.56', 'Comment.57', 'Comment.58']

  # filter non-NaN comments
  def filter_comments(row):
    return [comment for comment in row[comment_cols] if pd.notna(comment)]

  # Create a new column 'combined_comments' to store merged comments
  df['Comments'] = df.apply(filter_comments, axis=1)

  # Iterate through rows and build comment lists
  for index, row in df.iterrows():
    comments = [comment for comment in row[comment_cols] if pd.notnull(comment)]
    df.at[index, 'Comments'] = comments

  return df

df = merge_comments(df.copy())

df.head()


,Comment,Comment.1,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6,Comment.7,Comment.8,Comment.9,...,Comment.54,Comment.55,Comment.56,Comment.57,Comment.58,Environment,Summary,Description,embeddingsSummary,Comments
0,I am running the command provided by the docum...,,or the command “jf plugin install config-impor...,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,version we are running is: Artifactory Online ...,plugin config-import not found for jfrog Artif...,I am following the steps from [https://docs.jf...,"((tf.Tensor(0.03961963, shape=(), dtype=float3...",[I am running the command provided by the docu...
1,for Composer Remote repositories.{quote}\n\nRT...,it that relevant to the situation you described?,"does not support v2, only v1. We will check…",only supports v1. Can you explain the comment ...,Since Composer *Virtual* Repository supports o...,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,NaN,RTDEV-5069 / mirroring https://asset-packagist...,We are running Artifactory 7.63.14 and have pr...,"((tf.Tensor(0.008936056, shape=(), dtype=float...",[for Composer Remote repositories.{quote}\n\nR...
2,This is the current value I would like to chan...,,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,Developement and Production,Docker - Repository Default URL,Similar to Issue: [https://jfrog.atlassian.net...,"((tf.Tensor(0.020218397, shape=(), dtype=float...",[This is the current value I would like to cha...
3,},nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,Artifactory-pro 7.63.14 within a docker conta...,getting manifest unknown error pulling image,I am getting the following error when pulling ...,"((tf.Tensor(0.004147171, shape=(), dtype=float...","[}, nan, nan, nan, nan, nan, nan, nan, nan, na..."
4,"""PUT ‘REPO_PATH/s.zip’ HTTP/1.1"" 400 0 ""-"" ""py...",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,Linux centos,Caught exception while saving incoming stream ...,We have recently upgrade Artifactory to 7.63.1...,"((tf.Tensor(-0.024265409, shape=(), dtype=floa...","[""PUT ‘REPO_PATH/s.zip’ HTTP/1.1"" 400 0 ""-"" ""p..."


In [30]:
df['Comments'][700][0]

'Security General Settings :\n- Allow Anonymous Access\n'

In [32]:
data = []

# 2. Process each row with similarity above the threshold
for i, (summary, similarity) in enumerate(zip(df['Summary'], sum_similarities)):
    if similarity > 0.5:
        # new_df = new_df.append({
            data.append({
            'Summary': summary,
            'EmbeddingsSummary': embeddings[i],
            'Description': df.loc[i, 'Description'],
            'Comments': df.loc[i, 'Comments']
            })
        # }, ignore_index=True)

new_df = pd.DataFrame.from_records(data)

In [33]:
new_df.shape

(7, 4)

In [34]:
new_df.head()

,Summary,EmbeddingsSummary,Description,Comments
0,Invalid authentication removes groups from Cro...,"((tf.Tensor(-0.02690386, shape=(), dtype=float...",h3. Steps to reproduce\r\n # Configure Artifac...,[Closed this ticket as this is the desired beh...
1,Cannot log-in,"((tf.Tensor(-0.030607384, shape=(), dtype=floa...",Got internal error when trying to log-in. Trie...,[I just fixed it - it is caused by trying to h...
2,I would to use a certificate as an authenticat...,"((tf.Tensor(-0.02844619, shape=(), dtype=float...",Instead of just user credentials / api or toke...,"[No plans to implement at the moment., nan, na..."
3,Authentication problem,"((tf.Tensor(-0.014970663, shape=(), dtype=floa...","Hi,\nwe had some problems with our artifactory...",[Security General Settings :\n- Allow Anonymou...
4,Random authentification Issues,"((tf.Tensor(-0.04827584, shape=(), dtype=float...","Our Artifactory pro is using the LDAP plugin, ...",[We are having the exact same problem with LDA...


In [35]:
new_df['Description'] = new_df['Description'].astype(str)

In [36]:
embeddings = []

# Iterate through each row of the column
for i in new_df['Description']:
    # Encode the sentence using the USE model
    embedding = use_model([i])
    # Append the embedding to the list
    embeddings.append(embedding)

new_df['embeddingsDescription'] = embeddings

In [37]:
new_df["embeddingsDescription"] = new_df["embeddingsDescription"].tolist()

In [38]:
# Function to calculate cosine similarity between sentences
def calculate_similarity(input_sentence, embeddings):
  input_embedding = use_model([input_sentence])

  similarities = []
  for embedding in embeddings:
    similarity = cosine_similarity(input_embedding, embedding)[0][0]
    similarities.append(similarity)

  return similarities

In [39]:
# Calculate similarity scores
des_similarities = calculate_similarity(des_input_sen, new_df["embeddingsDescription"])

# Display results
for i, row in new_df.iterrows():
  Description = row["Description"]
  similarity = des_similarities[i]
  print(f"Description: {Description}, Similarity: {similarity:.4f}")

Description: h3. Steps to reproduce
 # Configure Artifactory to use Atlassian Crowd as authentication provider.
 # Import/sync some (Crowd) groups to Artifactory
 # Login to Artifactory web UI as a Crowd user with some (Crowd) group permissions that has been imported & configured to Artifactory
 # Verify that user details shows the groups associated for the user just logged in
 # Use another browser or browser private window for another session to Artifactory web UI, and make a new login to Artifactory, but now give wrong password

h3. Expected behavior

The second login fails but the first session remains with the same permissions.
I.e. failed login would not remove any groups & permissions.
h3. Actual behavior

The failed login triggers removal of all Crowd groups from the user and the permissions are lost.
Only re-login repopulates the groups & permissions to the user.
h3. Additional information

If authentication provider is LDAP (AD), failed login won't remove groups. Similar beha

In [40]:
len(des_similarities)

7

In [41]:
# Store descriptions with similarity > 0.7 and their scores in a list
similar_descriptions_and_scores = []
for Description, similarity in zip(new_df["Description"], des_similarities):
  if similarity > 0.5:
    similar_descriptions_and_scores.append((Description, similarity))

# Sort similar descriptions by similarity score (highest first)
sorted_similar_descriptions_and_scores = sorted(similar_descriptions_and_scores, key=lambda x: x[1], reverse=True)

# Print top 5 similar sentences and their scores
for i, (Description, similarity) in enumerate(sorted_similar_descriptions_and_scores):
  if i == 5:
    break  # Print only the top 5
  print(f"Similar Description: {Description} \n , Similarity: {similarity:.4f} \n")

Similar Description: Hi,
we had some problems with our artifactory when the user access a virtual repo which has an local repo who you need an authenticated user.


* virtual repo
**  local repo 1
**  local repo 2
**  external repo 1
** external repo 2
	

Permissions Management:
 - Any Remote (on any remote repo): anonymous -> deploy
 - local-repo1-permission (local repo 1): reader-user -> read, deployer-user -> delete, deploy, annotate, read
 - local-repo2-permission (local repo 2): reader-user -> read, deployer-user -> read

The problem occur when the needed jar is in "local repo 1" and your credential are from one of the both users 
 , Similarity: 1.0000 



In [42]:
len(similar_descriptions_and_scores)

1

In [44]:
similar_descriptions_and_scores[0]

('Hi,\nwe had some problems with our artifactory when the user access a virtual repo which has an local repo who you need an authenticated user.\n\n\n* virtual repo\n**  local repo 1\n**  local repo 2\n**  external repo 1\n** external repo 2\n\t\n\nPermissions Management:\n - Any Remote (on any remote repo): anonymous -> deploy\n - local-repo1-permission (local repo 1): reader-user -> read, deployer-user -> delete, deploy, annotate, read\n - local-repo2-permission (local repo 2): reader-user -> read, deployer-user -> read\n\nThe problem occur when the needed jar is in "local repo 1" and your credential are from one of the both users',
 1.0000001)

### Comments

In [45]:
model = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')

RuntimeError: Op type not registered 'SentencepieceOp' in binary running on 82ba1cc1b91b. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.

In [46]:
comments_for_similar_descriptions = []

for description, similarity in similar_descriptions_and_scores:

  # Find the index of the matching description in the original DataFrame
  description_index = new_df[new_df['Description'] == description].index[0]

  # Extract the corresponding comment
  comment = new_df.loc[description_index, 'Comments']

  # Append a tuple (description, similarity, comment) to the comments list
  comments_for_similar_descriptions.append((comment))

In [48]:
emb_coms = []
for comment in comments_for_similar_descriptions :
  emb_com = use_model(comment)
  emb_coms.append(emb_com)


In [55]:
query_embedding = model([des_input_sen])
cosines = []
# for i in emb_coms:
#   ss_score = util.semantic_search(query_embedding, i)
#   s_search.append(ss_score)

def semantic_search(query_embedding, corpus, document_ids=None):

  # Calculate cosine similarities between query and corpus embeddings
  for emb_com in emb_coms:
      cosine = cosine_similarity(query_embedding, emb_com)
      cosines.append(cosine)
  top_k_indices = tf.math.top_k(cosines, k=k).indices.numpy()  # Most similar indices
  top_k_similar_docs = [corpus[i] for i in top_k_indices.flatten()]
  top_k_distances = cosines.numpy().flatten()[top_k_indices.flatten()]  # Distances

  if document_ids is not None:
    top_k_document_ids = [document_ids[i] for i in top_k_indices.flatten()]
    return top_k_similar_docs, top_k_distances, top_k_document_ids
  else:
    return top_k_similar_docs, top_k_distances

# Example usage
k = 5  # Number of top similar documents to retrieve

top_similar_docs, top_distances = semantic_search(query_embedding, comments_for_similar_descriptions)
print("Top", k, "similar documents:")
for i, doc in enumerate(top_similar_docs):
  print(f"{i+1}. {doc} (distance: {top_distances[i]:.4f})")

IndexError: list index out of range